In [116]:
import numpy as np
from datasets import load_dataset
import re

import Embeddings.positional_embedding as pe
import feed_forward as ff
import Attention.attention_block as ab
import Attention.attention_head as ah
import transformer_block as tb
import layer_norm as ln
import base_transformer as bt

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Component parts of the transformer model

In [144]:
# test embedding
test_embedding = np.random.normal(0, 1, size=(8,9,1024))

In [135]:
# positional embeddings
test_pos_embed = pe.positional_embedding(1000, 1000)
test_pos_embed.embeddings.std()

0.020021235620774045

In [ ]:
# feed forward layer
test_layer = ff.neuron_layer(
      input_shape=512, output_shape=512
    , activation='relu', batch_size=8
    , clip_val=.0001, learning_rate=.001)

In [ ]:
# attention head
test_head = ah.attention_head(512, 64)
test_head.masked_attention_score(test_embedding).shape

In [ ]:
# attention block
test_block = ab.attention_block(num_heads=8, block_shape=512)
test_block.multi_head_attention(test_embedding).shape

(9, 512)

In [ ]:
# layer norm
test_layer_norm = ln.layer_norm(512)
test_layer_norm.layer_norm(test_embedding).shape

(9, 512)

In [ ]:
# transformer block
test_transformer_block = tb.transformer_block(
      num_heads=8, block_shape=512, activation='relu'
    , batch_size=8, clip_val=.0001, learning_rate=.001)
test_transformer_block.forward_pass(test_embedding).shape

### Transformer model

In [171]:
# test embedding
test_embedding = np.random.normal(0, 1, size=(4,9,1024))

In [188]:
# constants
START_TOKEN = '<START>'
END_TOKEN = '<END>'
NUM_SAMPLES = 10000
imdbDataset = load_dataset("stanfordnlp/imdb")

embeddingsFilepath = '/Users/josep/Desktop/Self/Learning/NLP/RNN/data/glove.6B.300d.txt'

# helper functions
def read_corpus(dataset):
    files = dataset["train"]["text"][:NUM_SAMPLES]
    return [[START_TOKEN] + [re.sub(r'[^\w]', '', w.lower()) for w in f.split(" ")] + [END_TOKEN] for f in files]


def embedding_for_vocab(filepath, words, dimensions):
    vocab_size = len(words)
    embeddings = np.zeros((vocab_size, dimensions))

    with open(filepath, encoding="utf8") as f:
        for line in f:
            word, *vector = line.split()
            if word in words.keys():
                index = words[word]
                embeddings[index] = np.array(vector)[:dimensions]
    return embeddings

imdbCorpus = read_corpus(imdbDataset)

corpusWords = [y for x in imdbCorpus for y in x]
corpusWords = list(set(corpusWords))
word2ind={}
for i in range(len(corpusWords)+1):
    word2ind[corpusWords[i-1]] = i
word2ind['<PAD>'] = 0
embeddings = embedding_for_vocab(embeddingsFilepath, word2ind, 300)

In [ ]:
# function to map words to indices
imdb_corpus_arrays = [np.array(text) for text in imdbCorpus]
def word_2_ind_func(x, word2ind):
    # for key, value in word2ind.items():
    #     print(value)
    #     texts[texts==key] = value
    return word2ind.get(x, x)
word_2_ind_func = np.vectorize(word_2_ind_func)#, otypes=[str])

# imdb_corpus_arrays_vectorized = word_2_ind_func(imdb_corpus_arrays)

In [317]:
# transformer model
test_transformer = bt.transformer(
      input_layer_shape=300, input_layer_activation='relu'
    , hidden_layer_shapes=[512,512,512], hidden_layer_activations=['relu', 'relu', 'relu']
    , hidden_layer_num_heads=[8, 8, 8]
    , output_shape=71083
    , output_layer_activation='softmax'
)
# test_transformer.input_layer.layer_weights.shape
# output = test_transformer.next_token_vocab_index(test_embedding)
# output

In [ ]:
# batching, mapping
batch_size = 4
batches = []
for text_num in range(0, len(imdbCorpus), batch_size):
    batch = imdb_corpus_arrays[text_num : text_num+batch_size]
    
    # finding max length in batch
    max_text_len = 0
    for text in batch:
        if len(text) > max_text_len:
            max_text_len = len(text)
    
    # padding
    padded_batch = []
    for text in batch:
        delta_from_max = max_text_len - len(text)
        if delta_from_max > 0:
            pad_array = np.full(delta_from_max, fill_value='<PAD>')
            text = np.concatenate((text, pad_array))
        padded_batch.append(text)
    
    # making array and converting to indexed
    batch_array = np.vstack(padded_batch)
    batch_array_ind = word_2_ind_func(batch_array, word2ind)

    # turning into embeddings
    batch_embeddings = embeddings[batch_array_ind]
    batches.append(batch_embeddings)
    # forward pass
    

In [324]:
for batch in batches:
    print(test_transformer.next_token_vocab_index(batch))

[16184 14823 40744 34470]
[43293 40744 17327 24446]
[15040  1175 34470 24818]


KeyboardInterrupt: 